In [1]:
%pylab inline
import pymc3 as pm
import pandas as pd
import scipy.stats as stats
import seaborn as sns
import theano.tensor as T
import theano
plt.style.use(['seaborn-talk'])
plt.rcParams["figure.figsize"] = (10,8)
print(pm.__version__)
print(theano.__version__)

Populating the interactive namespace from numpy and matplotlib
3.5
1.0.2


/home/deadhead/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
c_light ="#DCBCBC"
c_light_highlight ="#C79999"
c_mid ="#B97C7C"
c_mid_highlight ="#A25050"
c_dark ="#8F2727"
c_dark_highlight ="#7C0000"

In [6]:
generative_ensemble4 = pm.Model()

N = 1000
R = 1000

with generative_ensemble4:
    theta = pm.Beta(name="theta", alpha = 2.8663, beta = 2.8663)
    lambda_ = pm.InverseGamma(name="lambda", alpha = 3.48681, beta = 9.21604)
    BoundZeroInflatedPoisson = pm.Bound(pm.ZeroInflatedPoisson, lower=0, upper = 14)
    y = BoundZeroInflatedPoisson(name = "y", psi = theta, theta = lambda_, shape = (N,))

IndexError: tuple index out of range

In [18]:
import pymc3 as pm

example = pm.Model()

with example:
    BoundPoisson = pm.Bound(pm.Poisson, upper = 6)
    y = BoundPoisson(name = "y", mu = 1)

IndexError: tuple index out of range

In [ ]:
pm.Poisson(mu=)

In [12]:
with pm.Model() as model:
    BoundedNormal = pm.Bound(pm.Normal, lower=0.0)
    x = BoundedNormal('x', mu=1.0, sd=3.0)

In [19]:
import sys; sys.version

'3.6.2 |Continuum Analytics, Inc.| (default, Jul 20 2017, 13:51:32) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]'